In [ ]:
import random
import time
import numpy as np
from deap import base, creator, tools, algorithms

# Schwefel function
def schwefel_function(individual):
    return 418.9829 * len(individual) - sum([x * np.sin(np.sqrt(abs(x))) for x in individual]),

# Ограничение значений после мутации и кроссинговера
def check_bounds(individual, param_ranges):
    for i in range(len(individual)):
        individual[i] = max(param_ranges[f'x{i}'][0], min(individual[i], param_ranges[f'x{i}'][1]))
    return individual

# Настройка среды DEAP для генетического алгоритма
def genetic_algorithm_deap(param_ranges, population_size, generations, mutation_rate, crossover_rate, elitism_fraction, alpha, mu, sigma, tournsize):
    # Создание минимизирующей функции в DEAP
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Функция инициализации индивидов
    def create_individual():
        return creator.Individual([random.uniform(*param_ranges[param]) for param in param_ranges])

    # Настройка инструментария DEAP
    toolbox = base.Toolbox()
    toolbox.register("individual", create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", schwefel_function)
    toolbox.register("mate", tools.cxBlend, alpha=alpha)  # Параметр alpha для кроссинговера
    toolbox.register("mutate", tools.mutGaussian, mu=mu, sigma=sigma, indpb=mutation_rate)  # Параметры мутации
    toolbox.register("select", tools.selTournament, tournsize=tournsize)  # Размер турнира

    # Инициализация популяции
    population = toolbox.population(n=population_size)

    # Добавление элитизма
    elite_size = int(population_size * elitism_fraction)

    # Старт отсчета времени
    start_time = time.time()

    best_individual = None
    best_score = float('inf')
    best_generation = 0

    # Запуск алгоритма
    for gen in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)

        # Применение функции check_bounds ко всем потомкам
        offspring = [check_bounds(ind, param_ranges) for ind in offspring]

        fits = list(map(toolbox.evaluate, offspring))

        # Оценка лучших индивидов
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit
            if fit[0] < best_score:
                best_individual = ind
                best_score = fit[0]
                best_generation = gen + 1

        population = tools.selBest(offspring, elite_size) + tools.selBest(offspring, population_size - elite_size)

        print(f"Generation {gen + 1}, Best Score: {best_score}, Best Params: {best_individual}")

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Вывод результатов
    print("\nРезультаты оптимизации:")
    print(f"Оптимальная итерация: {best_generation}")
    print(f"Время до оптимальной итерации: {elapsed_time:.2f} секунд")
    print(f"Значение оптимальной функции: {best_score}")
    print(f"Оптимальные значения признаков: {best_individual}")

    return best_individual, best_score

# Пример использования

param_ranges = {
    "x0": (-500, 500),
    "x1": (-500, 500),
    "x2": (-500, 500),
    "x3": (-500, 500),
    "x4": (-500, 500),
    "x5": (-500, 500),
    "x6": (-500, 500),
    "x7": (-500, 500),
    "x8": (-500, 500),
    "x9": (-500, 500),
}

# Значимые гиперпараметры
population_size = 2000     # Размер популяции
generations = 1000        # Количество поколений
mutation_rate = 0.3      # Скорость мутации
crossover_rate = 0.2      # Вероятность кроссинговера
elitism_fraction = 0.3    # Доля элитизма
alpha = 0.9               # Параметр alpha для кроссинговера
mu = 0                    # Среднее значение для мутации
sigma = 0.5               # Стандартное отклонение для мутации
tournsize = 3             # Размер турнира

best_params, best_score = genetic_algorithm_deap(
    param_ranges=param_ranges,
    population_size=population_size,
    generations=generations,
    mutation_rate=mutation_rate,
    crossover_rate=crossover_rate,
    elitism_fraction=elitism_fraction,
    alpha=alpha,
    mu=mu,
    sigma=sigma,
    tournsize=tournsize
)


In [ ]:
import random
import time
import numpy as np
from deap import base, creator, tools, algorithms
import pandas as pd
from tqdm import tqdm
import gc

# Schwefel function
def schwefel_function(individual):
    return 418.9829 * len(individual) - sum([x * np.sin(np.sqrt(abs(x))) for x in individual]),

# Ограничение значений после мутации и кроссинговера
def check_bounds(individual, param_ranges):
    for i in range(len(individual)):
        individual[i] = max(param_ranges[f'x{i}'][0], min(individual[i], param_ranges[f'x{i}'][1]))
    return individual

# Настройка среды DEAP для генетического алгоритма
def genetic_algorithm_deap(param_ranges, population_size, generations, mutation_rate, crossover_rate,
                           elitism_fraction, alpha, mu, sigma, tournsize, crossover_method, mutation_method,
                           selection_method):
    # Создание минимизирующей функции в DEAP
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Функция инициализации индивидов
    def create_individual():
        return creator.Individual([random.uniform(*param_ranges[param]) for param in param_ranges])

    # Настройка инструментария DEAP
    toolbox = base.Toolbox()
    toolbox.register("individual", create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", schwefel_function)

    # Handling crossover strategies
    if crossover_method == tools.cxBlend:
        toolbox.register("mate", crossover_method, alpha=alpha)  # Only cxBlend uses alpha
    elif crossover_method == tools.cxUniform:
        toolbox.register("mate", crossover_method, indpb=0.5)  # cxUniform uses indpb
    else:
        toolbox.register("mate", crossover_method)  # Other crossover methods

    # Handling mutation strategies
    if mutation_method == tools.mutGaussian:
        toolbox.register("mutate", mutation_method, mu=mu, sigma=sigma, indpb=mutation_rate)  # Gaussian uses mu and sigma
    elif mutation_method == tools.mutUniformInt:
        low = [param_ranges[f"x{i}"][0] for i in range(len(param_ranges))]
        up = [param_ranges[f"x{i}"][1] for i in range(len(param_ranges))]
        toolbox.register("mutate", mutation_method, low=low, up=up, indpb=mutation_rate)
    else:
        toolbox.register("mutate", mutation_method, indpb=mutation_rate)  # Other mutation methods

    toolbox.register("select", selection_method, tournsize=tournsize)  # Tournament size

    # Инициализация популяции
    population = toolbox.population(n=population_size)

    # Добавление элитизма
    elite_size = int(population_size * elitism_fraction)

    # Старт отсчета времени
    start_time = time.time()

    best_individual = None
    best_score = float('inf')
    best_generation = 0
    best_generation_time = 0  # Track time until the best generation

    # Запуск алгоритма
    for gen in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)

        # Применение функции check_bounds ко всем потомкам
        offspring = [check_bounds(ind, param_ranges) for ind in offspring]

        fits = list(map(toolbox.evaluate, offspring))

        # Оценка лучших индивидов
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit
            if fit[0] < best_score:
                best_individual = ind
                best_score = fit[0]
                best_generation = gen + 1
                best_generation_time = time.time() - start_time

        population = tools.selBest(offspring, elite_size) + tools.selBest(offspring, population_size - elite_size)

        print(f"Generation {gen + 1}, Best Score: {best_score}, Best Params: {best_individual}")

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Вывод результатов
    print("\nOptimization results:")
    print(f"Best generation: {best_generation}")
    print(f"Time to best generation: {best_generation_time:.2f} seconds")
    print(f"Best function value: {best_score}")
    print(f"Best parameters: {best_individual}")

    return best_individual, best_score, best_generation, best_generation_time

# Генерация случайных гиперпараметров и стратегий
def generate_random_hyperparameters():
    crossover_methods = [tools.cxBlend, tools.cxTwoPoint, tools.cxUniform, tools.cxOnePoint]
    mutation_methods = [tools.mutGaussian, tools.mutFlipBit, tools.mutShuffleIndexes, tools.mutUniformInt]
    selection_methods = [tools.selTournament, tools.selRoulette, tools.selBest, tools.selRandom]

    hyperparameters = {
        "population_size": random.randint(100, 500),
        "mutation_rate": random.uniform(0.01, 0.5),
        "crossover_rate": random.uniform(0.01, 0.5),
        "elitism_fraction": random.uniform(0.01, 0.2),
        "mutation_percent_genes": random.uniform(0.01, 0.5),
        "mutation_by_replacement": random.choice([True, False]),
        "mutation_range_factor": random.uniform(0.01, 0.5),
        "alpha": random.uniform(0.01, 0.9),  # For crossover methods like Blend
        "mu": random.uniform(-1, 1),  # For Gaussian mutation
        "sigma": random.uniform(0.01, 1),  # For Gaussian mutation
        "tournsize": random.randint(2, 5),  # For tournament selection
        "crossover_method": random.choice(crossover_methods),
        "mutation_method": random.choice(mutation_methods),
        "selection_method": random.choice(selection_methods),
    }
    return hyperparameters

# Внешние и внутренние циклы
num_outer_runs = 30
num_inner_runs = 30
summary_results = []

param_ranges = {
    "x0": (-500, 500),
    "x1": (-500, 500),
    "x2": (-500, 500),
    "x3": (-500, 500),
    "x4": (-500, 500),
    "x5": (-500, 500),
    "x6": (-500, 500),
    "x7": (-500, 500),
    "x8": (-500, 500),
    "x9": (-500, 500),
}

# Внешний цикл
for outer_run in tqdm(range(num_outer_runs), desc="Outer Runs Progress"):
    print(f"\n=== Внешний прогон {outer_run + 1} ===\n")
    
    hyperparams = generate_random_hyperparameters()
    results = []

    # Внутренний цикл
    for inner_run in range(num_inner_runs):
        best_solution, best_fitness, best_generation, best_generation_time = genetic_algorithm_deap(
            param_ranges=param_ranges,
            population_size=hyperparams["population_size"],
            generations=500,  # Фиксированное количество поколений
            mutation_rate=hyperparams["mutation_rate"],
            crossover_rate=hyperparams["crossover_rate"],
            elitism_fraction=hyperparams["elitism_fraction"],
            alpha=hyperparams["alpha"],
            mu=hyperparams["mu"],
            sigma=hyperparams["sigma"],
            tournsize=hyperparams["tournsize"],
            crossover_method=hyperparams["crossover_method"],
            mutation_method=hyperparams["mutation_method"],
            selection_method=hyperparams["selection_method"]
        )

        run_data = {
            'Значение функции Швефеля': best_fitness,
            'Лучшая итерация': best_generation,
            'Время до лучшей итерации': best_generation_time
        }
        results.append(run_data)

    df_results = pd.DataFrame(results)
    median_fitness = df_results['Значение функции Швефеля'].median()
    mode_iteration = df_results['Лучшая итерация'].mode()[0]

    summary_row = {
        'Медиана значения функции Швефеля': median_fitness,
        'Мода лучшей итерации': mode_iteration,
        'Медиана времени до лучшей итерации': df_results['Время до лучшей итерации'].median(),
        'Crossover Method': hyperparams["crossover_method"].__name__,
        'Mutation Method': hyperparams["mutation_method"].__name__,
        'Selection Method': hyperparams["selection_method"].__name__,
        'Population Size': hyperparams["population_size"],
        'Mutation Rate': hyperparams["mutation_rate"],
        'Crossover Rate': hyperparams["crossover_rate"],
        'Elitism Fraction': hyperparams["elitism_fraction"],
        'Alpha': hyperparams["alpha"],
        'Mu': hyperparams["mu"],
        'Sigma': hyperparams["sigma"],
        'Tournament Size': hyperparams["tournsize"],
    }

    summary_results.append(summary_row)

    del df_results
    gc.collect()

# Итоговая таблица с результатами всех внешних прогонов
df_summary = pd.DataFrame(summary_results)
print("\nИтоговая таблица с медианными значениями и гиперпараметрами:")
print(df_summary)


In [ ]:
df_summary.to_csv('deap_shvefel.csv', index=False)